In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.decomposition import PCA
from scipy.spatial.distance import cosine

%load_ext autotime

time: 0 ns (started: 2021-05-22 04:19:49 +03:00)


# 1. Кастомные функции

## 1.1 Получить слова из текста в нужном формате

Берется файл с аналогиями, переводится в нужный вид, с которым будут работать в дальнейшем функции

In [2]:
def unique_list(l):
    ulist = []
    [ulist.append(x) for x in l if x not in ulist]
    return ulist

time: 0 ns (started: 2021-05-22 04:19:49 +03:00)


In [3]:
def get_analogy_words(filename):
    
    with open(filename, encoding='utf-8') as f:
        content = f.readlines()
    
    words_to_agg = [x.strip() for x in content] 
    
    words_to_agg = " ".join(words_to_agg)
    words_to_agg = " ".join(words_to_agg.split('_'))
    
    stop_words = ['PROPN', 'ADJ', 'VERB','ADV', 'NOUN']
    
    for i in stop_words:
        words_to_agg = words_to_agg.replace(i, '')
    
    words_to_agg = words_to_agg.replace('й','и')
    
    words_to_agg = ' '.join(unique_list(words_to_agg.split()))
    
    return words_to_agg

time: 0 ns (started: 2021-05-22 04:19:49 +03:00)


In [4]:
file1 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\столица-страна.txt'

anal1 = get_analogy_words(file1)

anal1

'афины греция багдад ирак бангкок таиланд пекин китаи берлин германия берн швеицария каир египет канберра австралия ханои вьетнам гавана куба хельсинки финляндия исламабад пакистан кабул афганистан лондон англия мадрид испания москва россия осло норвегия оттава канада париж франция рим италия стокгольм швеция тегеран иран токио япония'

time: 15 ms (started: 2021-05-22 04:19:49 +03:00)


## 1.2 Получить эмбеддинги ELMo

In [5]:
def get_embeddings_ELMo(model, sess, words_to_embed):
    
    embeddings = elmo([words_to_embed], signature="default", as_dict=True)["elmo"]
    
    words_embeddings = sess.run(embeddings)
    
    
    table_embedding = pd.DataFrame()
    for word_id in range(len(words_to_embed.lower().split())):
        table_embedding = table_embedding.append(pd.Series(words_embeddings[0][word_id]),ignore_index=True)

    table_embedding.index = words_to_embed.lower().split()
    
    return table_embedding
    

time: 0 ns (started: 2021-05-22 04:19:49 +03:00)


## 1.3 Решить задачу аналогии методом 3CosAdd

In [6]:
def solve_analogy_3CosAdd(words_to_embed, words_embeddings):
    
    index_list = np.arange(0,len(words_to_embed.split()))
    evensList = [x for x in index_list if x % 2 == 0]
    index_list =  np.arange(0,len(evensList))
    
    word_1 = [words_to_embed.lower().split()[i] for i in evensList]
    word_2 = [words_to_embed.lower().split()[i+1] for i in evensList]
    
    N_tasks = 0
    N_correct = 0
    
    for word_1_index in range(len(word_1)):
    
        #print('Looking for ', word_1[word_1_index])
    
        for pair_id in index_list:
            
            argmax = 0
            i_max = 0
            
            for word_vector in words_to_embed.lower().split():
            
                vecs_to_exclude = [word_2[word_1_index], word_1[pair_id], word_2[pair_id]]
                #print(vecs_to_exclude)
                #print(word_vector)
            
                #print(capital_id, pair_id)
                if ((word_vector not in vecs_to_exclude) & (word_1_index != pair_id)):
                
                    res = 1 - cosine(words_embeddings.loc[word_vector], (words_embeddings.loc[word_1[pair_id]] - words_embeddings.loc[word_2[pair_id]] + words_embeddings.loc[word_2[word_1_index]]))
                
                    #print(capitals[pair_id], '-', countries[pair_id], '+', countries[capital_id])
                    #print(res)
                    if (res > argmax):
                        argmax = res
                        i_max = word_vector
            N_tasks += 1
            
            #print(argmax)
        
            #print(word_1[pair_id], '-', word_2[pair_id], '+', word_2[word_1_index], '=', i_max)
        
        
            if (i_max == word_1[word_1_index]):
                N_correct += 1
        #print('')

    print('Accuracy score =', N_correct/N_tasks)
    print(N_correct)
    print(N_tasks)
    
    score = N_correct/N_tasks
    
    return score

time: 0 ns (started: 2021-05-22 04:19:49 +03:00)


## 1.4 Решить задачу аналогии методом 3CosAvg

In [7]:
def solve_analogy_3CosAvg(words_to_embed, words_embeddings):
    
    index_list = np.arange(0,len(words_to_embed.split()))
    evensList = [x for x in index_list if x % 2 == 0]
    index_list =  np.arange(0,len(evensList))
    
    word_1 = [words_to_embed.lower().split()[i] for i in evensList]
    word_2 = [words_to_embed.lower().split()[i+1] for i in evensList]
    
    N_tasks = 0
    N_correct = 0
    
    v_1 = 0
    v_2 = 0

    for pair_id in index_list:
        v_1 += words_embeddings.loc[word_1[pair_id]]
        v_2 += words_embeddings.loc[word_2[pair_id]]

    d = (v_1 - v_2)/len(index_list)
    
    N_tasks = 0
    N_correct = 0

    for word_2_index in range(len(word_2)):
        #print('looking for ', word_1[word_2_index])
        
        argmax = 0
        i_max = 0
        
        for word_vector in words_to_embed.lower().split():
            
            vecs_to_exclude = [word_2[word_2_index]]
            
            if (word_vector not in vecs_to_exclude):
                
                res = 1 - cosine(words_embeddings.loc[word_vector], (words_embeddings.loc[word_2[word_2_index]] + d))
                
                if (res > argmax):
                    argmax = res
                    i_max = word_vector
                    
            
            
        #print(word_2[word_2_index], '- d = ', i_max)
        N_tasks +=1
    
        if (i_max == word_1[word_2_index]):
            N_correct += 1
    
    print('Accuracy score =', N_correct/N_tasks)
    print(N_correct)
    print(N_tasks)
    
    score = N_correct/N_tasks
    
    return score

time: 0 ns (started: 2021-05-22 04:19:49 +03:00)


# 2. Решим задачи аналогий

## 2.1 Модель Elmo Ru News

In [8]:
elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz",trainable=True)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

time: 6.66 s (started: 2021-05-21 13:40:20 +03:00)


### 2.1.1 Прилагательные - антонимы с приставкой не-

In [16]:
file1 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\adj-NOTadj.txt'

analogy_1 = get_analogy_words(file1)

analogy_1

'приемлемыи неприемлемыи простои непростои определенныи неопределенныи ясныи неясныи удобныи неудобныи конкурентноспособныи неконкурентноспособныи последовательныи непоследовательныи убедительныи неубедительныи решенныи нерешенныи эффективныи неэффективныи этичныи неэтичныи удачныи неудачныи честныи нечестныи уместныи неуместныи информативныи неинформативныи осведомленныи неосведомленныи известныи неизвестныи вероятныи невероятныи логичныи нелогичныи приятныи неприятныи возможныи невозможныи продуктивныи непродуктивныи рациональныи иррациональныи разумныи неразумныи ответственныи безответственныи уверенныи неуверенныи благодарныи неблагодарныи'

time: 16 ms (started: 2021-05-21 15:18:59 +03:00)


In [10]:
analogy_1_embeddings = get_embeddings_ELMo(elmo,sess,analogy_1)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 1min 12s (started: 2021-05-20 22:23:31 +03:00)


In [11]:
analogy_1_embeddings.to_csv('elmo_ru_news_analogy1_emb', sep='\t')

time: 50.5 s (started: 2021-05-20 22:24:47 +03:00)


In [12]:
analogy_1_score = solve_analogy_3CosAdd(analogy_1,analogy_1_embeddings)

Accuracy score = 0.9053497942386831
660
729
time: 52.2 s (started: 2021-05-20 22:25:37 +03:00)


In [13]:
analogy_1_score_3CosAvg = solve_analogy_3CosAvg(analogy_1,analogy_1_embeddings)

Accuracy score = 0.9629629629629629
26
27
time: 688 ms (started: 2021-05-20 22:26:29 +03:00)


### 2.1.2 Страна - прилагательное

In [17]:
file2 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\country-adjective.txt'

analogy_2 = get_analogy_words(file2)

analogy_2

'албания албанскии аргентина аргентинскии австралия австралиискии австрия австриискии беларусь белорусскии бразилия бразильскии болгария болгарскии камбоджа камбоджиискии чили чилиискии китаи китаискии колумбия колумбиискии хорватия хорватскии дания датскии египет египетскии англия англиискии франция французскии германия немецкии греция греческии исландия исландскии индия индиискии ирландия ирландскии израиль израильскии италия итальянскии япония японскии корея кореискии македония македонскии мальта мальтиискии мексика мексиканскии молдова молдавскии нидерланды голландскии норвегия норвежскии перу перуанскии польша польскии португалия португальскии россия русскии словакия словацкии испания испанскии швеция шведскии швеицария швеицарскии таиланд таискии украина украинскии'

time: 31 ms (started: 2021-05-21 15:19:02 +03:00)


In [15]:
analogy_2_embeddings = get_embeddings_ELMo(elmo,sess,analogy_2)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 5min 44s (started: 2021-05-20 22:26:30 +03:00)


In [16]:
analogy_2_embeddings.to_csv('elmo_ru_news_analogy2_emb', sep='\t')

time: 4.78 s (started: 2021-05-20 22:32:26 +03:00)


In [17]:
analogy_2_score = solve_analogy_3CosAdd(analogy_2,analogy_2_embeddings)

Accuracy score = 0.594883997620464
1000
1681
time: 1min 50s (started: 2021-05-20 22:32:31 +03:00)


In [18]:
analogy_2_score_3CosAvg = solve_analogy_3CosAvg(analogy_2,analogy_2_embeddings)

Accuracy score = 0.6829268292682927
28
41
time: 1.59 s (started: 2021-05-20 22:34:21 +03:00)


### 2.1.3 Страна - язык

In [18]:
file3 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\country-language.txt'

analogy_3 = get_analogy_words(file3)

analogy_3

'андорра каталанскии аргентина испанскии австралия англиискии австрия немецкии багамы бангладеш бенгальскии барбадос белиз боливия бразилия португальскии камбоджа кхмерскии канада чили колумбия куба кипр греческии дания датскии египет арабскии эфиопия амхарскии фиджи гваделупа французскии гватемала гаиана иран персидскии иордания кувеит мозамбик никарагуа норвегия норвежскии палестина перу сирия таивань китаискии сша венесуэла россия русскии'

time: 15 ms (started: 2021-05-21 15:19:05 +03:00)


In [20]:
analogy_3_embeddings = get_embeddings_ELMo(elmo,sess,analogy_3)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 7min 36s (started: 2021-05-20 22:34:23 +03:00)


In [21]:
analogy_3_embeddings.to_csv('elmo_ru_news_analogy3_emb', sep='\t')

time: 4.41 s (started: 2021-05-20 22:42:06 +03:00)


In [22]:
analogy_3_score = solve_analogy_3CosAdd(analogy_3,analogy_3_embeddings)

Accuracy score = 0.17455621301775148
118
676
time: 27 s (started: 2021-05-20 22:42:12 +03:00)


In [23]:
analogy_3_score_3CosAvg = solve_analogy_3CosAvg(analogy_3,analogy_3_embeddings)

Accuracy score = 0.19230769230769232
5
26
time: 688 ms (started: 2021-05-20 22:42:39 +03:00)


### 2.1.4 Страна - валюта

In [19]:
file4 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\country-money.txt'

analogy_4 = get_analogy_words(file4)

analogy_4

'алжир динар ангола кванза аргентина песо армения драм бразилия реал болгария лев камбоджа риель канада доллар хорватия куна дания крона европа евро венгрия форинт индия рупия иран риал япония иена корея вона латвия лат литва лит македония денар малаизия ринггит мексика нигерия наира польша злотыи румыния леи россия рубль швеция таиланд бат украина гривна сша вьетнам донг'

time: 16 ms (started: 2021-05-21 15:19:07 +03:00)


In [20]:
analogy_4 = analogy_4.replace('сша','')
analogy_4 = analogy_4.replace('мексика','')
analogy_4 = analogy_4.replace('швеция','')

time: 0 ns (started: 2021-05-21 15:19:20 +03:00)


In [26]:
analogy_4_embeddings = get_embeddings_ELMo(elmo,sess,analogy_4)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 20min 39s (started: 2021-05-20 22:42:40 +03:00)


In [27]:
analogy_4_embeddings.to_csv('elmo_ru_news_analogy4_emb', sep='\t')

time: 3.38 s (started: 2021-05-20 23:03:26 +03:00)


In [28]:
analogy_4_score = solve_analogy_3CosAdd(analogy_4,analogy_4_embeddings)

Accuracy score = 0.4513031550068587
329
729
time: 30.2 s (started: 2021-05-21 01:53:38 +03:00)


In [29]:
analogy_4_score_3CosAvg = solve_analogy_3CosAvg(analogy_4,analogy_4_embeddings)

Accuracy score = 0.7037037037037037
19
27
time: 781 ms (started: 2021-05-21 01:54:08 +03:00)


### 2.1.5 Сущ. единственное число - множественное число

In [21]:
file5 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\singular-plural.txt'

analogy_5 = get_analogy_words(file5)

analogy_5

'улыбка улыбки число числа завод заводы ящик ящики дисковод дисководы период периоды день дни песня песни вебсаит вебсаиты бог боги принцип принципы автомобиль автомобили правительство правительства бумага бумаги препарат препараты эксперт эксперты факт факты дорога дороги событие события офис офисы наука науки альбом альбомы задача задачи игрок игроки контакт контакты штат штаты плакат плакаты государство государства нагреватель нагреватели кружка кружки вещь вещи сезон сезоны друг друзья ягодица ягодицы налог налоги студент студенты система системы книга книги рыба рыбы операция операции ночь ночи понятие понятия запас запасы желание желания билет билеты приказ приказы коллега коллеги артист артисты преступление преступления покупатель покупатели закон законы популяция популяции пользователь пользователи жертва жертвы инженер инженеры версия версии мольберт мольберты пример примеры механизм механизмы цена цены язык языки академия академии водоем водоемы предмет предметы неделя недели

time: 94 ms (started: 2021-05-21 15:19:23 +03:00)


In [31]:
analogy_5_embeddings = get_embeddings_ELMo(elmo,sess,analogy_5)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 9min 1s (started: 2021-05-21 01:54:09 +03:00)


In [32]:
analogy_5_embeddings.to_csv('elmo_ru_news_analogy5_emb', sep='\t')

time: 906 ms (started: 2021-05-21 02:03:12 +03:00)


In [33]:
analogy_5_score = solve_analogy_3CosAdd(analogy_5,analogy_5_embeddings)

Accuracy score = 0.9879
9879
10000
time: 26min 23s (started: 2021-05-21 02:03:13 +03:00)


In [34]:
analogy_5_score_3CosAvg = solve_analogy_3CosAvg(analogy_5,analogy_5_embeddings)

Accuracy score = 1.0
100
100
time: 9.48 s (started: 2021-05-21 02:29:37 +03:00)


### 2.1.6 Глагол - существительное с -тель

In [22]:
file6 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\verb-noun.txt'

analogy_6 = get_analogy_words(file6)

analogy_6

'исследовать исследователь слушать слушатель водить водитель нанимать наниматель крушить крушитель мучить мучитель грабить грабитель спасать спасатель карать каратель читать читатель строить строитель искать искатель мыслить мыслитель угнетать угнетатель почитать почитатель заклинать заклинатель предпринимать предприниматель получать получатель отправлять отправитель истязать истязатель держать держатель воздыхать воздыхатель учить учитель вершить вершитель глотать глотатель разжигать разжигатель воспевать воспеватель потакать потакатель писать писатель деять деятель созидать созидатель преподавать преподаватель покрывать покрыватель душить душитель ваять ваятель кормить кормитель служить служитель открывать открыватель надзирать надзиратель следовать следователь владеть владетель добывать добыватель вымогать вымогатель завещать завещатель изобретать изобретатель крестить креститель носить носитель содержать содержатель копать копатель обитать обитатель обладать обладатель поджигать по

time: 31 ms (started: 2021-05-21 15:19:25 +03:00)


In [10]:
analogy_6_embeddings = get_embeddings_ELMo(elmo,sess,analogy_6)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 2min 38s (started: 2021-05-21 10:05:57 +03:00)


In [11]:
analogy_6_embeddings.to_csv('elmo_ru_news_analogy6_emb', sep='\t')

time: 328 ms (started: 2021-05-21 10:08:35 +03:00)


In [12]:
analogy_6_score = solve_analogy_3CosAdd(analogy_6,analogy_6_embeddings)

Accuracy score = 0.9036989795918368
2834
3136
time: 4min 58s (started: 2021-05-21 10:08:36 +03:00)


In [13]:
analogy_6_score_3CosAvg = solve_analogy_3CosAvg(analogy_6,analogy_6_embeddings)

Accuracy score = 0.9464285714285714
53
56
time: 3.23 s (started: 2021-05-21 10:13:35 +03:00)


### 2.1.7 Глагол - существительное с -ение

In [23]:
file7 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\verb-noun_3.txt'

analogy_7 = get_analogy_words(file7)

analogy_7

'обвинять обвинение восхищать восхищение утверждать утверждение вычислять вычисление продолжить продолжение объявлять объявление лишать лишение наблюдать наблюдение вдохновлять вдохновление освещать освещение воображать воображение определять определение восстановить восстановление истекать истечение приложить приложение истреблять истребление возбуждать возбуждение выступать выступление гореть горение дозволять дозволение достигать достижение завершать завершение задымлять задымление засорять засорение изготовлять изготовление измельчать измельчение крушить крушение лечить лечение назначать назначение ознакомлять ознакомление осведомлять осведомление очищать очищение повреждать повреждение повышать повышение пополнять пополнение поручать поручение поселять поселение посещать посещение потрясать потрясение предвкушать предвкушение представлять представление преуменьшать преуменьшение примирять примирение разрушать разрушение располагать расположение рассуждать рассуждение свергать свер

time: 32 ms (started: 2021-05-21 15:19:27 +03:00)


In [15]:
analogy_7_embeddings = get_embeddings_ELMo(elmo,sess,analogy_7)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 11min 48s (started: 2021-05-21 10:13:38 +03:00)


In [16]:
analogy_7_embeddings.to_csv('elmo_ru_news_analogy7_emb', sep='\t')

time: 10.2 s (started: 2021-05-21 10:25:35 +03:00)


In [17]:
analogy_7_score = solve_analogy_3CosAdd(analogy_7,analogy_7_embeddings)

Accuracy score = 0.9808264462809917
2967
3025
time: 4min 47s (started: 2021-05-21 10:25:49 +03:00)


In [18]:
analogy_7_score_3CosAvg = solve_analogy_3CosAvg(analogy_7,analogy_7_embeddings)

Accuracy score = 1.0
55
55
time: 3.11 s (started: 2021-05-21 10:30:36 +03:00)


### 2.1.8 Глагол - существительное с -ация

In [24]:
file8 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\verb_noun_2.txt'

analogy_8 = get_analogy_words(file8)

analogy_8

'колонизировать колонизация компилировать компиляция конфигурировать конфигурация консультировать консультация персонализировать персонализация глобализировать глобализация импровизировать импровизация максимизировать маскимизация минимизировать минимизация модернизировать модернизация оптимизировать оптимизация организовывать организация приватизировать приватизация рандомизировать рандомизация стабилизировать стабилизация стандартизировать стандартизация утилизировать утилизация визуализировать визуализация декларировать декларация монетизировать монетизация аппроксимировать аппроксимация интегрировать интеграция концентрировать концентрация урбанизировать урбанизация автоматизировать автоматизация авторизировать авторизация агитировать агитация адаптировать адаптация актуализировать актуализация ассоциировать ассоциация гиперболизировать гиперболизация деградировать деградация дегустировать дегустация дезинформировать дезинформация дезориентировать дезориентация декорировать декорац

time: 31 ms (started: 2021-05-21 15:19:30 +03:00)


In [20]:
analogy_8_embeddings = get_embeddings_ELMo(elmo,sess,analogy_8)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 5min 14s (started: 2021-05-21 10:30:40 +03:00)


In [21]:
analogy_8_embeddings.to_csv('elmo_ru_news_analogy8_emb', sep='\t')

time: 4.73 s (started: 2021-05-21 10:36:01 +03:00)


In [22]:
analogy_8_score = solve_analogy_3CosAdd(analogy_8,analogy_8_embeddings)

Accuracy score = 0.9421487603305785
2850
3025
time: 4min 43s (started: 2021-05-21 10:36:07 +03:00)


In [23]:
analogy_8_score_3CosAvg = solve_analogy_3CosAvg(analogy_8,analogy_8_embeddings)

Accuracy score = 1.0
55
55
time: 3.17 s (started: 2021-05-21 10:40:50 +03:00)


### 2.1.9 Мужской род - женский род

In [25]:
file9 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\мужской род - женский род.txt'

analogy_9 = get_analogy_words(file9)

analogy_9

'актер актриса мальчик девочка брат сестра бык корова бизнесмен бизнесвумен председатель председательница папа мама герцог герцогиня император императрица лис лисица бог богиня дедушка бабушка внук внучка жених невеста руководитель руководительница наследник наследница герои героиня муж жена король королева лев львица мужчина женщина директор директриса племянник племянница поэт поэтесса полицеискии полицеиская принц принцесса баран овца петух курица сын дочь жеребец кобыла тигр тигрица дядя тетя официант официантка вор воровка конь лошадь юноша девушка старик старуха супруг супруга красавец красавица певец певица тесть теща отчим мачеха работник работница школьник школьница виновник виновница учитель учительница писатель писательница ткач ткачиха артист артистка студент студентка чемпион чемпионка колдун колдунья журналист журналистка тракторист трактористка разведчик разведчица воспитатель воспитательница гимнаст гимнастка рекордсмен рекордсменка теннисист теннисистка бегун бегунья в

time: 32 ms (started: 2021-05-21 15:19:32 +03:00)


In [25]:
analogy_9_embeddings = get_embeddings_ELMo(elmo,sess,analogy_9)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 18min 36s (started: 2021-05-21 10:40:53 +03:00)


In [26]:
analogy_9_embeddings.to_csv('elmo_ru_news_analogy9_emb', sep='\t')

time: 8.25 s (started: 2021-05-21 10:59:56 +03:00)


In [27]:
analogy_9_score = solve_analogy_3CosAdd(analogy_9,analogy_9_embeddings)

Accuracy score = 0.9402985074626866
4221
4489
time: 8min 31s (started: 2021-05-21 11:00:04 +03:00)


In [28]:
analogy_9_score_3CosAvg = solve_analogy_3CosAvg(analogy_9,analogy_9_embeddings)

Accuracy score = 0.9552238805970149
64
67
time: 4.97 s (started: 2021-05-21 11:08:36 +03:00)


### 2.1.10 Прилагательное - наречие

In [26]:
file10 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\прилагательное-наречие_2.txt'

analogy_10 = get_analogy_words(file10)

analogy_10

'удивительныи удивительно очевидныи очевидно спокоиныи спокоино веселыи весело страшныи страшно эффективныи эффективно красивыи красиво свободныи свободно яростныи яростно счастливыи счастливо непосредственныи непосредственно нечастыи нечасто удачныи удачно полезныи полезно случаиныи случаино возможныи возможно точныи точно профессиональныи профессионально быстрыи быстро тихии тихо редкии редко неохотныи неохотно безопасныи безопасно серьезныи серьезно медленныи медленно внезапныи внезапно стремительныи стремительно типичныи типично несчастныи несчастно обычныи обычно'

time: 16 ms (started: 2021-05-21 15:19:35 +03:00)


In [30]:
analogy_10_embeddings = get_embeddings_ELMo(elmo,sess,analogy_10)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 16min 24s (started: 2021-05-21 11:08:42 +03:00)


In [31]:
analogy_10_embeddings.to_csv('elmo_ru_news_analogy10_emb', sep='\t')

time: 5.36 s (started: 2021-05-21 11:25:45 +03:00)


In [32]:
analogy_10_score = solve_analogy_3CosAdd(analogy_10,analogy_10_embeddings)

Accuracy score = 0.9655555555555555
869
900
time: 46.3 s (started: 2021-05-21 11:25:55 +03:00)


In [ ]:
analogy_10_score_3CosAvg = solve_analogy_3CosAvg(analogy_10,analogy_10_embeddings)

### 2.1.11 Прилагательное - сравнительная степень

In [27]:
file11 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\прилагательное-сравнительная степень.txt'

analogy_11 = get_analogy_words(file11)

analogy_11

'яркии ярче дешевыи дешевле холодныи холоднее крутои круче глубокии глубже легкии легче быстрыи быстрее тяжелыи тяжелее высокии выше жаркии жарче длинныи длиннее громкии громче низкии ниже безопасныи безопаснее острыи острее короткии короче простои проще медленныи медленнее умныи умнее сильныи сильнее крепкии крепче теплыи теплее слабыи слабее широкии шире молодои моложе'

time: 16 ms (started: 2021-05-21 15:19:38 +03:00)


In [34]:
analogy_11_embeddings = get_embeddings_ELMo(elmo,sess,analogy_11)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 19min 52s (started: 2021-05-21 11:26:43 +03:00)


In [35]:
analogy_11_embeddings.to_csv('elmo_ru_news_analogy11_emb', sep='\t')

time: 2.47 s (started: 2021-05-21 11:46:38 +03:00)


In [36]:
analogy_11_score = solve_analogy_3CosAdd(analogy_11,analogy_11_embeddings)

Accuracy score = 0.8784
549
625
time: 32.5 s (started: 2021-05-21 11:46:41 +03:00)


In [37]:
analogy_11_score_3CosAvg = solve_analogy_3CosAvg(analogy_11,analogy_11_embeddings)

Accuracy score = 0.92
23
25
time: 703 ms (started: 2021-05-21 11:47:14 +03:00)


### 2.1.12 Известная столица - страна

In [28]:
file12 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\столица-страна.txt'

analogy_12 = get_analogy_words(file12)

analogy_12

'афины греция багдад ирак бангкок таиланд пекин китаи берлин германия берн швеицария каир египет канберра австралия ханои вьетнам гавана куба хельсинки финляндия исламабад пакистан кабул афганистан лондон англия мадрид испания москва россия осло норвегия оттава канада париж франция рим италия стокгольм швеция тегеран иран токио япония'

time: 0 ns (started: 2021-05-21 15:19:40 +03:00)


In [39]:
analogy_12_embeddings = get_embeddings_ELMo(elmo,sess,analogy_12)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 26min 35s (started: 2021-05-21 11:47:15 +03:00)


In [40]:
analogy_12_embeddings.to_csv('elmo_ru_news_analogy12_emb', sep='\t')

time: 1.55 s (started: 2021-05-21 12:13:51 +03:00)


In [41]:
analogy_12_score = solve_analogy_3CosAdd(analogy_12,analogy_12_embeddings)

Accuracy score = 0.3799621928166352
201
529
time: 19.4 s (started: 2021-05-21 12:13:53 +03:00)


In [42]:
analogy_12_score_3CosAvg = solve_analogy_3CosAvg(analogy_12,analogy_12_embeddings)

Accuracy score = 0.5652173913043478
13
23
time: 594 ms (started: 2021-05-21 12:14:13 +03:00)


### 2.1.13 Любая столица - страна

In [9]:
file13 = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\столица-страна_2.txt'

analogy_13 = get_analogy_words(file13)

analogy_13

'абуджа нигерия аккра гана алжир амман иордания анкара турция антананариву мадагаскар апиа самоа ашхабад туркменистан асмэра эритрея астана казахстан афины греция багдад ирак баку азербаиджан бамако мали бангкок таиланд банжул гамбия пекин китаи беирут ливан белград сербия бельмопан белиз берлин германия берн швеицария бишкек кыргызстан братислава словакия брюссель бельгия бухарест румыния будапешт венгрия бужумбура бурунди каир египет канберра австралия каракас венесуэла кишинев молдова конакри гвинея копенгаген дания дакар сенегал дамаск сирия дакка бангладеш доха катар дублин ирландия душанбе таджикистан фунафути тувалу габороне ботсвана джорджтаун гаиана ханои вьетнам хараре зимбабве гавана куба хельсинки финляндия исламабад пакистан джакарта индонезия кабул афганистан кампала уганда катманду непал хартум судан киев украина кигали руанда кингстон ямаика либревиль габон лилонгве малави лима перу лиссабон португалия любляна словения лондон англия луанда ангола лусака замбия мадрид ис

time: 47 ms (started: 2021-05-21 13:40:49 +03:00)


In [10]:
analogy_13_embeddings = get_embeddings_ELMo(elmo,sess,analogy_13)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


time: 2min 34s (started: 2021-05-21 13:40:58 +03:00)


In [11]:
analogy_13_embeddings.to_csv('elmo_ru_news_analogy13_emb', sep='\t')

time: 39 s (started: 2021-05-21 13:43:32 +03:00)


In [12]:
analogy_13_score = solve_analogy_3CosAdd(analogy_13,analogy_13_embeddings)

Accuracy score = 0.14268431001890358
1887
13225
time: 44min 5s (started: 2021-05-21 13:44:11 +03:00)


In [13]:
analogy_13_score_3CosAvg = solve_analogy_3CosAvg(analogy_13,analogy_13_embeddings)

Accuracy score = 0.24347826086956523
28
115
time: 13.4 s (started: 2021-05-21 14:28:17 +03:00)


### Сохраним результаты по Elmo Ru News

#### Пересчетаем те, которые модель "забыла" 

Последний раз, она считала только по 13 аналогии, поэтому остальные нужно будет пересчитать

In [45]:
from os import listdir
from os.path import isfile, join

mypath = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles = onlyfiles

time: 0 ns (started: 2021-05-21 15:31:56 +03:00)


In [60]:
elmo_ru_news_scores_3CosAdd = []
elmo_ru_news_scores_3CosAvg = []

for i in tqdm(range(1,13)):
    file_name = 'elmo_ru_news_analogy'+str(i)+'_emb'
    file_words = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\'+onlyfiles[i]

    analogy_words = get_analogy_words(file_words)
    
    if (i==4):
        analogy_words = analogy_words.replace('сша','')
        analogy_words = analogy_words.replace('мексика','')
        analogy_words = analogy_words.replace('швеция','')
    
    analogy_embeddings = pd.read_csv(file_name, sep='\t')
    analogy_embeddings.index = analogy_embeddings['Unnamed: 0']
    analogy_embeddings = analogy_embeddings.drop(columns=['Unnamed: 0'])
    
    analogy_score_3CosAdd = solve_analogy_3CosAdd(analogy_words,analogy_embeddings)
    analogy_score_3CosAvg = solve_analogy_3CosAvg(analogy_words,analogy_embeddings)
    
    elmo_ru_news_scores_3CosAdd.append(analogy_score_3CosAdd)
    elmo_ru_news_scores_3CosAvg.append(analogy_score_3CosAvg)
    
    print('3CosAdd:')
    print(elmo_ru_news_scores_3CosAdd)
    
    print('3CosAvg:')
    print(elmo_ru_news_scores_3CosAvg)
    
elmo_ru_news_scores_3CosAdd.append(analogy_13_score)
elmo_ru_news_scores_3CosAvg.append(analogy_13_score_3CosAvg)


  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

Accuracy score = 0.9053497942386831
660
729


  8%|██████▉                                                                            | 1/12 [00:30<05:32, 30.22s/it]

Accuracy score = 0.9629629629629629
26
27
3CosAdd:
[0.9053497942386831]
3CosAvg:
[0.9629629629629629]
Accuracy score = 0.594883997620464
1000
1681


 17%|█████████████▊                                                                     | 2/12 [02:19<12:48, 76.84s/it]

...

Accuracy score = 1.0
30
30
3CosAdd:
[0.9053497942386831, 0.594883997620464, 0.17455621301775148, 0.4513031550068587, 0.9879, 0.9036989795918368, 0.9808264462809917, 0.9421487603305785, 0.9402985074626866, 0.9655555555555555]
3CosAvg:
[0.9629629629629629, 0.6829268292682927, 0.19230769230769232, 0.7037037037037037, 1.0, 0.9464285714285714, 1.0, 1.0, 0.9552238805970149, 1.0]
Accuracy score = 0.8784
549
625


 92%|██████████████████████████████████████████████████████████████████████████▎      | 11/12 [53:05<03:29, 209.64s/it]

Accuracy score = 0.92
23
25
3CosAdd:
[0.9053497942386831, 0.594883997620464, 0.17455621301775148, 0.4513031550068587, 0.9879, 0.9036989795918368, 0.9808264462809917, 0.9421487603305785, 0.9402985074626866, 0.9655555555555555, 0.8784]
3CosAvg:
[0.9629629629629629, 0.6829268292682927, 0.19230769230769232, 0.7037037037037037, 1.0, 0.9464285714285714, 1.0, 1.0, 0.9552238805970149, 1.0, 0.92]
Accuracy score = 0.3799621928166352
201
529


100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [53:22<00:00, 266.91s/it]

Accuracy score = 0.5652173913043478
13
23
3CosAdd:
[0.9053497942386831, 0.594883997620464, 0.17455621301775148, 0.4513031550068587, 0.9879, 0.9036989795918368, 0.9808264462809917, 0.9421487603305785, 0.9402985074626866, 0.9655555555555555, 0.8784, 0.3799621928166352]
3CosAvg:
[0.9629629629629629, 0.6829268292682927, 0.19230769230769232, 0.7037037037037037, 1.0, 0.9464285714285714, 1.0, 1.0, 0.9552238805970149, 1.0, 0.92, 0.5652173913043478]
time: 53min 22s (started: 2021-05-21 16:22:19 +03:00)


In [65]:
elmo_ru_news_scores_3CosAdd = [0.9053497942386831, 0.594883997620464, 
                               0.17455621301775148, 0.4513031550068587, 
                               0.9879, 0.9036989795918368, 
                               0.9808264462809917, 0.9421487603305785, 
                               0.9402985074626866, 0.9655555555555555, 
                               0.8784, 0.3799621928166352]

elmo_ru_news_scores_3CosAdd.append(analogy_13_score)

elmo_ru_news_scores_3CosAdd

[0.9053497942386831,
 0.594883997620464,
 0.17455621301775148,
 0.4513031550068587,
 0.9879,
 0.9036989795918368,
 0.9808264462809917,
 0.9421487603305785,
 0.9402985074626866,
 0.9655555555555555,
 0.8784,
 0.3799621928166352,
 0.14268431001890358]

time: 500 ms (started: 2021-05-21 17:38:10 +03:00)


In [69]:
pd.Series(elmo_ru_news_scores_3CosAdd).to_csv('elmo_ru_news_3CosAdd_results', sep='\t')

time: 94 ms (started: 2021-05-21 17:44:57 +03:00)


In [70]:
elmo_ru_news_scores_3CosAvg = [0.9629629629629629, 0.6829268292682927, 
                               0.19230769230769232, 0.7037037037037037, 
                               1.0, 0.9464285714285714, 
                               1.0, 1.0, 
                               0.9552238805970149, 1.0, 
                               0.92, 0.5652173913043478]

elmo_ru_news_scores_3CosAvg.append(analogy_13_score_3CosAvg)

elmo_ru_news_scores_3CosAvg

[0.9629629629629629,
 0.6829268292682927,
 0.19230769230769232,
 0.7037037037037037,
 1.0,
 0.9464285714285714,
 1.0,
 1.0,
 0.9552238805970149,
 1.0,
 0.92,
 0.5652173913043478,
 0.24347826086956523]

time: 15 ms (started: 2021-05-21 17:44:57 +03:00)


In [19]:
elmo_ru_news_scores_3CosAvg = [0.9629629629629629,
 0.6829268292682927,
 0.19230769230769232,
 0.7037037037037037,
 1.0,
 0.9464285714285714,
 1.0,
 1.0,
 0.9552238805970149,
 1.0,
 0.92,
 0.5652173913043478,
 0.24347826086956523]

time: 0 ns (started: 2021-05-22 15:08:18 +03:00)


In [20]:
pd.Series(elmo_ru_news_scores_3CosAvg).to_csv('elmo_ru_news_3CosAvg_results', sep='\t')

time: 0 ns (started: 2021-05-22 15:08:23 +03:00)


## 2.2 Модель Elmo Ru Wiki

In [8]:
elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz",trainable=True)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

time: 8.47 s (started: 2021-05-21 23:46:32 +03:00)


### Напишем цикл, который идет по всем аналогиям сам

In [8]:
from os import listdir
from os.path import isfile, join

mypath = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles = onlyfiles

time: 31 ms (started: 2021-05-22 04:27:01 +03:00)


In [10]:
for analogy_number in tqdm(range(10,14)):
    
    file_words = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\'+onlyfiles[analogy_number]
    
    analogy_words = get_analogy_words(file_words)
    
    if (analogy_number==4):
        analogy_words = analogy_words.replace('сша','')
        analogy_words = analogy_words.replace('мексика','')
        analogy_words = analogy_words.replace('швеция','')
        
    analogy_embeddings = get_embeddings_ELMo(elmo,sess,analogy_words)
    
    file_name = 'elmo_ru_wiki_analogy'+str(analogy_number)+'_emb'
    analogy_embeddings.to_csv(file_name, sep='\t')

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
 25%|█████████████████████                                                               | 1/4 [00:15<00:47, 15.92s/it]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
 50%|█████████████████████████████████████████▌                                         | 2/4 [08:59<10:27, 313.79s/it]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
 75%|██████████████████████████████████████████████████████████████▎                    | 3/4 [19:51<07:48, 468.66s/it]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [35:32<00:00, 533.24s/it]


time: 35min 35s (started: 2021-05-21 23:46:40 +03:00)


### Решим проблему аналогий

In [9]:
elmo_ru_wiki_scores_3CosAdd = []
elmo_ru_wiki_scores_3CosAvg = []

for i in tqdm(range(1,14)):
    file_name = 'elmo_ru_wiki_analogy'+str(i)+'_emb'
    file_words = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\'+onlyfiles[i]

    analogy_words = get_analogy_words(file_words)
    
    if (i==4):
        analogy_words = analogy_words.replace('сша','')
        analogy_words = analogy_words.replace('мексика','')
        analogy_words = analogy_words.replace('швеция','')
    
    analogy_embeddings = pd.read_csv(file_name, sep='\t')
    analogy_embeddings.index = analogy_embeddings['Unnamed: 0']
    analogy_embeddings = analogy_embeddings.drop(columns=['Unnamed: 0'])
    
    analogy_score_3CosAdd = solve_analogy_3CosAdd(analogy_words,analogy_embeddings)
    analogy_score_3CosAvg = solve_analogy_3CosAvg(analogy_words,analogy_embeddings)
    
    elmo_ru_wiki_scores_3CosAdd.append(analogy_score_3CosAdd)
    elmo_ru_wiki_scores_3CosAvg.append(analogy_score_3CosAvg)
    
    print('3CosAdd:')
    print(elmo_ru_wiki_scores_3CosAdd)
    
    print('3CosAvg:')
    print(elmo_ru_wiki_scores_3CosAvg)
    

  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

Accuracy score = 0.8916323731138546
650
729


  8%|██████▍                                                                            | 1/13 [00:31<06:12, 31.06s/it]

Accuracy score = 0.9259259259259259
25
27
3CosAdd:
[0.8916323731138546]
3CosAvg:
[0.9259259259259259]
Accuracy score = 0.6686496133254015
1124
1681


 15%|████████████▊                                                                      | 2/13 [02:20<14:08, 77.15s/it]

...

Accuracy score = 0.92
23
25
3CosAdd:
[0.8916323731138546, 0.6686496133254015, 0.21301775147928995, 0.5171467764060357, 0.9835, 0.9655612244897959, 0.9735537190082645, 0.9629752066115702, 0.9376253063042994, 0.9666666666666667, 0.8464]
3CosAvg:
[0.9259259259259259, 0.7560975609756098, 0.2692307692307692, 0.5925925925925926, 0.99, 1.0, 1.0, 0.9818181818181818, 0.9552238805970149, 1.0, 0.92]
Accuracy score = 0.5614366729678639
297
529


 92%|██████████████████████████████████████████████████████████████████████████▊      | 12/13 [53:21<02:33, 153.01s/it]

Accuracy score = 0.6956521739130435
16
23
3CosAdd:
[0.8916323731138546, 0.6686496133254015, 0.21301775147928995, 0.5171467764060357, 0.9835, 0.9655612244897959, 0.9735537190082645, 0.9629752066115702, 0.9376253063042994, 0.9666666666666667, 0.8464, 0.5614366729678639]
3CosAvg:
[0.9259259259259259, 0.7560975609756098, 0.2692307692307692, 0.5925925925925926, 0.99, 1.0, 1.0, 0.9818181818181818, 0.9552238805970149, 1.0, 0.92, 0.6956521739130435]
Accuracy score = 0.16763705103969753
2217
13225


100%|███████████████████████████████████████████████████████████████████████████████| 13/13 [1:34:40<00:00, 436.97s/it]

Accuracy score = 0.21739130434782608
25
115
3CosAdd:
[0.8916323731138546, 0.6686496133254015, 0.21301775147928995, 0.5171467764060357, 0.9835, 0.9655612244897959, 0.9735537190082645, 0.9629752066115702, 0.9376253063042994, 0.9666666666666667, 0.8464, 0.5614366729678639, 0.16763705103969753]
3CosAvg:
[0.9259259259259259, 0.7560975609756098, 0.2692307692307692, 0.5925925925925926, 0.99, 1.0, 1.0, 0.9818181818181818, 0.9552238805970149, 1.0, 0.92, 0.6956521739130435, 0.21739130434782608]
time: 1h 34min 40s (started: 2021-05-22 04:27:15 +03:00)


In [10]:
pd.Series(elmo_ru_wiki_scores_3CosAdd).to_csv('elmo_ru_wiki_3CosAdd_results', sep='\t')

time: 16 ms (started: 2021-05-22 06:01:56 +03:00)


In [18]:
pd.Series(elmo_ru_wiki_scores_3CosAvg).to_csv('elmo_ru_wiki_3CosAvg_results', sep='\t')

time: 15 ms (started: 2021-05-22 15:07:11 +03:00)


## 2.3 Модель Elmo Ru Twitter

In [8]:
elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-twitter_2013-01_2018-04_600k_steps.tar.gz",trainable=True)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

time: 6.81 s (started: 2021-05-22 04:11:09 +03:00)


### Напишем цикл, который идет по всем аналогиям сам

In [9]:
from os import listdir
from os.path import isfile, join

mypath = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles = onlyfiles

time: 0 ns (started: 2021-05-22 04:11:16 +03:00)


In [10]:
for analogy_number in tqdm(range(12,14)):
    
    file_words = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\'+onlyfiles[analogy_number]
    
    analogy_words = get_analogy_words(file_words)
    
    if (analogy_number==4):
        analogy_words = analogy_words.replace('сша','')
        analogy_words = analogy_words.replace('мексика','')
        analogy_words = analogy_words.replace('швеция','')
        
    analogy_embeddings = get_embeddings_ELMo(elmo,sess,analogy_words)
    
    file_name = 'elmo_ru_twitter_analogy'+str(analogy_number)+'_emb'
    analogy_embeddings.to_csv(file_name, sep='\t')

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
 50%|██████████████████████████████████████████                                          | 1/2 [00:23<00:23, 23.55s/it]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [06:37<00:00, 198.90s/it]

time: 6min 37s (started: 2021-05-22 04:11:19 +03:00)


### Решим проблему аналогий

In [12]:
elmo_ru_twitter_scores_3CosAdd = []
elmo_ru_twitter_scores_3CosAvg = []

for i in tqdm(range(1,14)):
    file_name = 'elmo_ru_twitter_analogy'+str(i)+'_emb'
    file_words = 'C:\\Users\\Asus\\Data analysis\\Учеба\\ДИПЛОМ\\Analogies\\Small\\'+onlyfiles[i]

    analogy_words = get_analogy_words(file_words)
    
    if (i==4):
        analogy_words = analogy_words.replace('сша','')
        analogy_words = analogy_words.replace('мексика','')
        analogy_words = analogy_words.replace('швеция','')
    
    analogy_embeddings = pd.read_csv(file_name, sep='\t')
    analogy_embeddings.index = analogy_embeddings['Unnamed: 0']
    analogy_embeddings = analogy_embeddings.drop(columns=['Unnamed: 0'])
    
    analogy_score_3CosAdd = solve_analogy_3CosAdd(analogy_words,analogy_embeddings)
    analogy_score_3CosAvg = solve_analogy_3CosAvg(analogy_words,analogy_embeddings)
    
    elmo_ru_twitter_scores_3CosAdd.append(analogy_score_3CosAdd)
    elmo_ru_twitter_scores_3CosAvg.append(analogy_score_3CosAvg)
    
    print('3CosAdd:')
    print(elmo_ru_twitter_scores_3CosAdd)
    
    print('3CosAvg:')
    print(elmo_ru_twitter_scores_3CosAvg)
    

  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

Accuracy score = 0.9465020576131687
690
729


  8%|██████▍                                                                            | 1/13 [00:30<06:09, 30.77s/it]

Accuracy score = 1.0
27
27
3CosAdd:
[0.9465020576131687]
3CosAvg:
[1.0]
Accuracy score = 0.711481261154075
1196
1681


 15%|████████████▊                                                                      | 2/13 [02:20<14:09, 77.20s/it]

...

Accuracy score = 1.0
25
25
3CosAdd:
[0.9465020576131687, 0.711481261154075, 0.14644970414201183, 0.41838134430727025, 0.9895, 0.9534438775510204, 0.9626446280991735, 0.9312396694214876, 0.9501002450434395, 0.9666666666666667, 0.9584]
3CosAvg:
[1.0, 0.8536585365853658, 0.11538461538461539, 0.4444444444444444, 1.0, 1.0, 1.0, 0.9818181818181818, 0.9850746268656716, 1.0, 1.0]
Accuracy score = 0.3931947069943289
208
529


 92%|██████████████████████████████████████████████████████████████████████████▊      | 12/13 [53:29<02:33, 153.23s/it]

Accuracy score = 0.5217391304347826
12
23
3CosAdd:
[0.9465020576131687, 0.711481261154075, 0.14644970414201183, 0.41838134430727025, 0.9895, 0.9534438775510204, 0.9626446280991735, 0.9312396694214876, 0.9501002450434395, 0.9666666666666667, 0.9584, 0.3931947069943289]
3CosAvg:
[1.0, 0.8536585365853658, 0.11538461538461539, 0.4444444444444444, 1.0, 1.0, 1.0, 0.9818181818181818, 0.9850746268656716, 1.0, 1.0, 0.5217391304347826]
Accuracy score = 0.07742911153119092
1024
13225


100%|███████████████████████████████████████████████████████████████████████████████| 13/13 [1:35:12<00:00, 439.40s/it]

Accuracy score = 0.12173913043478261
14
115
3CosAdd:
[0.9465020576131687, 0.711481261154075, 0.14644970414201183, 0.41838134430727025, 0.9895, 0.9534438775510204, 0.9626446280991735, 0.9312396694214876, 0.9501002450434395, 0.9666666666666667, 0.9584, 0.3931947069943289, 0.07742911153119092]
3CosAvg:
[1.0, 0.8536585365853658, 0.11538461538461539, 0.4444444444444444, 1.0, 1.0, 1.0, 0.9818181818181818, 0.9850746268656716, 1.0, 1.0, 0.5217391304347826, 0.12173913043478261]
time: 1h 35min 12s (started: 2021-05-22 06:01:56 +03:00)


In [13]:
pd.Series(elmo_ru_twitter_scores_3CosAdd).to_csv('elmo_ru_twitter_3CosAdd_results', sep='\t')

time: 16 ms (started: 2021-05-22 07:37:08 +03:00)


In [14]:
pd.Series(elmo_ru_twitter_scores_3CosAvg).to_csv('elmo_ru_twitter_3CosAvg_results', sep='\t')

time: 16 ms (started: 2021-05-22 07:37:08 +03:00)
